In [1]:
!pip install evaluate


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from IPython.display import display

In [ ]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer
from tqdm import tqdm
from datasets import Dataset
import pickle
import random
import evaluate


In [ ]:
import wandb
wandb.login(key="c6f0c2b20da326f7cd01857887a897f898ba5b4f")

In [4]:
from huggingface_hub import notebook_login

notebook_login()

In [12]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
# train_df = pd.read_csv('/kaggle/input/dataset/train.csv')
# test_df = pd.read_csv('/kaggle/input/dataset/test.csv')
train_df.head()

,ID,set,question,answer,score
0,0,Q2,Hamtube เป็นแพลตฟอร์มดูวีดีโอออนไลน์ ที่อนุญาต...,granularity ควรจะมีค่าต่ำ เพราะว่าเราต้องการทร...,0.0
1,1,Q3,Hamtube เป็นแพลตฟอร์มดูวีดีโอออนไลน์ ที่อนุญาต...,เห็นด้วย เพราะเป็นการเก็บข้อมูลจากหลาย users ...,5.0
2,2,Q2,Hamtube เป็นแพลตฟอร์มดูวีดีโอออนไลน์ ที่อนุญาต...,granularity ควรเป็น #checkout events/ #cookies...,5.0
3,3,Q3,Hamtube เป็นแพลตฟอร์มดูวีดีโอออนไลน์ ที่อนุญาต...,"เห็นด้วย ให้ X~Binomial(N,p) โดย p เป็นอัตราส่...",2.0
4,4,Q1,Hamtube เป็นแพลตฟอร์มดูวีดีโอออนไลน์ ที่อนุญาต...,เห็นด้วย เนื่องจากการทดสอบ A/B Testing เป็นวิธ...,4.5


In [13]:
train_df.describe()

,ID,score
count,362.000000,362.000000
mean,180.500000,3.021409
std,104.644637,1.990644
min,0.000000,0.000000
25%,90.250000,1.000000
50%,180.500000,3.500000
75%,270.750000,5.000000
max,361.000000,5.000000


In [14]:
train_df["text"] = f"Q: {train_df['question']} A: {train_df['answer']}"
train_df.drop(columns=["question", "answer", "set"], inplace=True)

test_df["text"] = f"Q: {test_df['question']} A: {test_df['answer']}"
test_df.drop(columns=["question", "answer", "set"], inplace=True)

train_df.head()

,ID,score,text
0,0,0.0,Q: 0 Hamtube เป็นแพลตฟอร์มดูวีดีโอออนไลน์...
1,1,5.0,Q: 0 Hamtube เป็นแพลตฟอร์มดูวีดีโอออนไลน์...
2,2,5.0,Q: 0 Hamtube เป็นแพลตฟอร์มดูวีดีโอออนไลน์...
3,3,2.0,Q: 0 Hamtube เป็นแพลตฟอร์มดูวีดีโอออนไลน์...
4,4,4.5,Q: 0 Hamtube เป็นแพลตฟอร์มดูวีดีโอออนไลน์...


In [15]:
test_df.head()

,ID,text
0,362,Q: 0 Hamtube เป็นแพลตฟอร์มดูวีดีโอออนไลน์ ...
1,363,Q: 0 Hamtube เป็นแพลตฟอร์มดูวีดีโอออนไลน์ ...
2,364,Q: 0 Hamtube เป็นแพลตฟอร์มดูวีดีโอออนไลน์ ...
3,365,Q: 0 Hamtube เป็นแพลตฟอร์มดูวีดีโอออนไลน์ ...
4,366,Q: 0 Hamtube เป็นแพลตฟอร์มดูวีดีโอออนไลน์ ...


In [17]:
train_df['score'].value_counts().shape

(14,)

In [12]:
train_data = train_df.copy().to_numpy()
test_data = test_df.copy().to_numpy()

label_2_idx = dict(zip(contacts, range(len(contacts))))
idx_2_label = dict(zip(range(len(contacts)), contacts))

print("Create Mappings")
display(idx_2_label)
display(label_2_idx)

# print("Before Mappings")
# display(data[:, 1])
# data[:,1] = np.vectorize(label_2_num_map.get)(data[:,1])
# print("After Mappings")
# display(data[:, 1])

Create Mappings


{0: 'สำนักงานตำรวจแห่งชาติ',
 1: 'การรถไฟฟ้าขนส่งมวลชนแห่งประเทศไทย',
 2: 'สภาเด็กและเยาวชนกรุงเทพมหานคร',
 3: 'กรมควบคุมมลพิษ',
 4: 'กรมสรรพสามิต',
 5: 'การไฟฟ้านครหลวง',
 6: 'กรมทางหลวง',
 7: 'สำนักงานประกันสุขภาพแห่งชาติ',
 8: 'การประปานครหลวง',
 9: 'คณะกรรมการการพัฒนาเศรษฐกิจ',
 10: 'กระทรวงการท่องเที่ยวและกีฬา',
 11: 'สำนักงาน กสทช. ศูนย์รับแจ้งปัญหา 1200'}

{'สำนักงานตำรวจแห่งชาติ': 0,
 'การรถไฟฟ้าขนส่งมวลชนแห่งประเทศไทย': 1,
 'สภาเด็กและเยาวชนกรุงเทพมหานคร': 2,
 'กรมควบคุมมลพิษ': 3,
 'กรมสรรพสามิต': 4,
 'การไฟฟ้านครหลวง': 5,
 'กรมทางหลวง': 6,
 'สำนักงานประกันสุขภาพแห่งชาติ': 7,
 'การประปานครหลวง': 8,
 'คณะกรรมการการพัฒนาเศรษฐกิจ': 9,
 'กระทรวงการท่องเที่ยวและกีฬา': 10,
 'สำนักงาน กสทช. ศูนย์รับแจ้งปัญหา 1200': 11}

In [13]:
def set_random_seed(seed: int):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
    
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_random_seed(42)

In [26]:
model_name = "airesearch/wangchanberta-base-att-spm-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name, revision='main', model_max_length=512)

num_labels = 14
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=num_labels,
)

Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at airesearch/wangchanberta-base-att-spm-uncased and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [21]:
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=256)

train_dataset = Dataset.from_pandas(train_df)
train_dataset = train_dataset.map(tokenize_function, batched=True)
train_dataset = train_dataset.rename_column("score", "labels")

test_dataset = Dataset.from_pandas(test_df)
test_dataset = test_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/362 [00:00<?, ? examples/s]

Map:   0%|          | 0/90 [00:00<?, ? examples/s]

In [24]:
train_dataset

Dataset({
    features: ['ID', 'labels', 'text', 'input_ids', 'attention_mask'],
    num_rows: 362
})

In [25]:
test_dataset

Dataset({
    features: ['ID', 'text', 'input_ids', 'attention_mask'],
    num_rows: 90
})

In [ ]:
training_args = TrainingArguments(
    output_dir="cem_text_classification_wangchanberta",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    report_to="wandb"
)

In [ ]:
clf_metrics = evaluate.combine(["accuracy", "f1", "precision", "recall", "mse"])

def sigmoid(x):
   return 1/(1 + np.exp(-x))

def compute_metrics(eval_pred):

   predictions, labels = eval_pred
   predictions = sigmoid(predictions)
   predictions = (predictions > 0.5).astype(int).reshape(-1)
   return clf_metrics.compute(predictions=predictions, references=labels.astype(int).reshape(-1))

In [21]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

<ipython-input-21-217194651456>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [22]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.066400,0.240364,0.927706,0.000000,0.000000,0.000000
2,0.055600,0.255033,0.925632,0.000000,0.000000,0.000000
3,0.049600,0.264103,0.925971,0.000000,0.000000,0.000000


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
 

TrainOutput(global_step=9591, training_loss=0.06604290530574967, metrics={'train_runtime': 4840.0292, 'train_samples_per_second': 63.402, 'train_steps_per_second': 1.982, 'total_flos': 2.018683738824192e+16, 'train_loss': 0.06604290530574967, 'epoch': 3.0})

In [23]:
import shutil

shutil.make_archive("/kaggle/working/checkpoint-9591", "zip", "/kaggle/working/multi_label_text_classification_wangchanberta_model/checkpoint-9591")


'/kaggle/working/checkpoint-9591.zip'

In [24]:
predictions = trainer.predict(test_dataset)

/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [25]:
logits = torch.tensor(predictions.predictions)
probs = torch.sigmoid(logits)
pred_labels = (probs > 0.5).int().tolist()
pred_df = pd.DataFrame(pred_labels, columns=contacts)

test_df = pd.read_csv('/kaggle/input/dataset/test.csv')
output_df = pd.concat([test_df["id"], pred_df], axis=1)

In [26]:
output_df.to_csv('submission.csv', index=False)

In [27]:
output_df

,id,สำนักงานตำรวจแห่งชาติ,การรถไฟฟ้าขนส่งมวลชนแห่งประเทศไทย,สภาเด็กและเยาวชนกรุงเทพมหานคร,กรมควบคุมมลพิษ,กรมสรรพสามิต,การไฟฟ้านครหลวง,กรมทางหลวง,สำนักงานประกันสุขภาพแห่งชาติ,การประปานครหลวง,คณะกรรมการการพัฒนาเศรษฐกิจ,กระทรวงการท่องเที่ยวและกีฬา,สำนักงาน กสทช. ศูนย์รับแจ้งปัญหา 1200
0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,1,0,0,0
2,2,0,0,0,0,0,0,0,0,0,0,0,0
3,3,1,0,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10805,10805,0,0,0,0,0,0,0,0,0,1,0,0
10806,10806,1,0,0,0,0,0,0,0,0,0,0,0
10807,10807,0,0,0,0,0,0,0,0,0,1,0,0
10808,10808,1,0,0,0,0,0,0,0,0,0,0,0
